In [2]:
import pandas as pd
import os
import numpy as np 
import random

import random
import pandas as pd
import numpy as np
import pickle
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

# Read Data 

In [3]:
raw_path = './raw/'
olist_rcm = pd.read_csv(raw_path + 'olist_recommendation_dataset.csv')


# Data overview

In [4]:
olist_rcm.head()

,Customer State,Review Score,Purchase Timestamp,Purchase Date,Monetary,Customer Segment,Marketing Action,Customer_ID,Order_ID,Product_ID,product_name,aisle,department,Product Category
0,RJ,5,9/13/2017 8:59,9/13/2017 0:00,281.30,Potential Loyalists,Cross Sell Recommendations and Discount coupons,15438,0,8446,Utility Lighter,more household,household,more household
1,GO,5,6/28/2017 11:52,6/28/2017 0:00,73.86,Lost Customers,Don't spend too much trying to re-acquire,22529,7229,8446,Utility Lighter,more household,household,more household
2,MG,4,5/18/2018 10:25,5/18/2018 0:00,108.32,Potential Loyalists,Cross Sell Recommendations and Discount coupons,17139,31508,8446,Utility Lighter,more household,household,more household
3,PR,5,8/1/2017 18:38,8/1/2017 0:00,472.06,Loyal Customers,Loyality programs;Cross Sell,33008,42252,8446,Utility Lighter,more household,household,more household
4,MG,5,8/10/2017 21:48,8/10/2017 0:00,1415.74,VVIP - Can't Loose Them,No Price Incentives; Offer Limited edition and...,41377,47321,8446,Utility Lighter,more household,household,more household


In [5]:
features = ['Customer_ID','product_name','Product_ID','Review Score']
olist_rcm = olist_rcm[features]
olist_rcm = olist_rcm.rename(columns={"Review Score":"ratings","Customer_ID":"customer_id","Product_ID":"product_id"})

In [6]:
from scipy.sparse import csr_matrix

In [7]:
olist_rcm = olist_rcm[:20000]

In [8]:
olist_rcm.customer_id.unique()

array([15438, 22529, 17139, ..., 24740, 11139, 37746], dtype=int64)

In [9]:
# pivot ratings into movie features
df_product_features = olist_rcm.reset_index().pivot_table(
    index='customer_id',
    columns='product_id',
    values='ratings'
).fillna(0)
# convert dataframe of movie features to scipy sparse matrix
mat_product_features = csr_matrix(df_product_features.values)

In [10]:
df_product_features.head()

product_id,63,69,151,161,185,219,226,238,255,311,...,32021,32050,32052,32083,32115,32134,32164,32170,32171,32203
customer_id,,,,,,,,,,,,,,,,,,,,,
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Buil Recommendations system

In [11]:
df_product_features.shape

(15530, 1170)

In [12]:
X = df_product_features.T
X.head()

customer_id,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
product_id,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
X.index[:-20]

Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,   311,
       ...
       31567, 31575, 31585, 31605, 31607, 31631, 31654, 31668, 31671, 31692],
      dtype='int64', name='product_id', length=1150)

In [14]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(1170, 10)

In [15]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(1170, 1170)

In [16]:
correlation_matrix

array([[ 1.        , -0.78765498, -0.69339966, ..., -0.26517297,
        -0.02805237, -0.56961809],
       [-0.78765498,  1.        ,  0.73663496, ...,  0.39473083,
         0.02148933,  0.58777927],
       [-0.69339966,  0.73663496,  1.        , ...,  0.61717909,
        -0.4557374 ,  0.68701627],
       ...,
       [-0.26517297,  0.39473083,  0.61717909, ...,  1.        ,
        -0.14667544,  0.0475913 ],
       [-0.02805237,  0.02148933, -0.4557374 , ..., -0.14667544,
         1.        , -0.18022077],
       [-0.56961809,  0.58777927,  0.68701627, ...,  0.0475913 ,
        -0.18022077,  1.        ]])

In [17]:
fave_prod = olist_rcm.groupby(['customer_id']).max()['product_id'].to_frame()

In [18]:
fave_prod = fave_prod.reset_index()
fave_prod

,customer_id,product_id
0,5,10630
1,9,6927
2,12,15675
3,15,27919
4,16,10630
...,...,...
15525,41419,13331
15526,41421,18573
15527,41422,15250
15528,41424,21641


In [19]:
def get_prod_id(customer_id):
    
    product_id = fave_prod[fave_prod.customer_id == customer_id]['product_id']
    return product_id
    
    
prd_id = get_prod_id(41422)
product_id = prd_id.iloc[0]
product_id

15250

In [20]:
product_names = list(X.index)
product_ID_Idx = product_names.index(product_id)
product_ID_Idx

529

In [21]:

correlation_product_ID = correlation_matrix[product_ID_Idx]
correlation_product_ID.shape

(1170,)

In [22]:
Recommend = list(X.index[correlation_product_ID > 0.70])
# Removes the item already bought by the customer
Recommend.remove(product_id) 
Recommend[0:20]

[69,
 602,
 1477,
 3196,
 3417,
 4075,
 5821,
 6475,
 8600,
 9499,
 9621,
 12089,
 13206,
 15434,
 16043,
 16929,
 16961,
 18584,
 20536,
 20885]

In [23]:
## Getting Product names froom prediction 

predictions = pd.DataFrame(Recommend[:20])
predictions.columns = ['product_id']
predictions

,product_id
0,69
1,602
2,1477
3,3196
4,3417
5,4075
6,5821
7,6475
8,8600
9,9499


In [24]:
predictions['Product Name'] = predictions.product_id.apply(lambda x : olist_rcm[olist_rcm.product_id == x]['product_name'].unique()[0])


In [25]:
predictions.head(10)

,product_id,Product Name
0,69,Vanilla with Almond Milk Iced Coffee
1,602,Banana Chocolate Chip
2,1477,Wild Mushroom Cauliflower Hempseed Burgers
3,3196,Healthy Volume Shampoo
4,3417,Chocolate Creme Cookies
5,4075,White Asparagus Peeled Spears
6,5821,Asian Noodles Teriyaki
7,6475,Limonata Sparkling Lemon Beverage
8,8600,Meat Loaf 30% Less Sodium Seasoning Mix
9,9499,Thai Peanut Sauce


In [31]:
olist_rcm.to_csv(raw_path + 'olist_ALS_dataset.csv', index=False)


In [32]:
X.to_csv('X_ALS.csv')

In [33]:
correlation_matrix.shape

(1170, 1170)

In [34]:
import json

with open('correlation_matrix_ALS.txt', 'w') as filehandle:
    json.dump(correlation_matrix.tolist(), filehandle)

In [36]:
olist_rcm = pd.read_csv(raw_path +'olist_ALS_dataset.csv')


In [38]:
X = pd.read_csv('X_ALS.csv', index_col=0)
X

,5,9,12,15,16,19,20,24,27,29,...,41410,41412,41413,41416,41418,41419,41421,41422,41424,41425
product_id,,,,,,,,,,,,,,,,,,,,,
63,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
151,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
X.columns[4000:4020]

Index(['10675', '10677', '10679', '10680', '10681', '10682', '10685', '10686',
       '10689', '10694', '10696', '10698', '10699', '10700', '10703', '10712',
       '10713', '10714', '10716', '10719'],
      dtype='object')

In [40]:
import json
with open('correlation_matrix_ALS.txt') as f:
    correlation_matrix = json.load(f)

In [41]:
correlation_matrix = np.array(correlation_matrix)

In [42]:
X.index

Index([   63,    69,   151,   161,   185,   219,   226,   238,   255,   311,
       ...
       32021, 32050, 32052, 32083, 32115, 32134, 32164, 32170, 32171, 32203],
      dtype='int64', name='product_id', length=1170)

In [43]:
def Recommendations_ALS(customer_id):
    #Olist_db_ = Olist_db_[Olist_db_.Customer_ID < 3600]
    fave_prod = olist_rcm.groupby(['customer_id']).max()['product_id'].to_frame()
    fave_prod = fave_prod.reset_index()

    #prd_id = get_prod_id(Customer_id)
    prd_id = fave_prod[fave_prod.customer_id == customer_id]['product_id']
    product_id = prd_id.iloc[0]
    product_names = list(X.index)
    product_ID_Idx = product_names.index(product_id)

    correlation_product_ID = correlation_matrix[product_ID_Idx]
    
    Recommend = list(X.index[correlation_product_ID > 0.70])
    # Removes the item already bought by the customer
    Recommend.remove(product_id) 
    
    ## Getting Product names froom prediction 

    predictions = pd.DataFrame(Recommend[:20])
    predictions.columns = ['product_id']

    predictions['Product Name'] = predictions.product_id.apply(lambda x : olist_rcm[olist_rcm.product_id == x]['product_name'].unique()[0])
    Recommendations = predictions[:10]
    return Recommendations

In [44]:
Recommendations = Recommendations_ALS(10000)
Recommendations[:10]

,product_id,Product Name
0,1969,Plain Cream Cheese
1,2637,Colby Pepper Jack Snack Bites
2,3366,"Soup Mix, Sausage & Lentil, Indiana Harvest"
3,3756,Blueberry Sparkling Water
4,3959,Signature Twin Size Mattress
5,4599,California Pinot Noir Red Wine
6,5828,Powder Fresh Roll-On Antiperspirant Deodorant
7,6305,Toasted Sesame Crackers
8,7927,No Touch Hand Soap Unit
9,8061,Blueberry Sparkling Water


In [46]:
Recommendations = Recommendations_ALS(2000)
Recommendations[:10]

,product_id,Product Name
0,219,Snakku
1,226,Shaved Parmesan Cheese
2,238,Alpha-Bits Multigrain Cereal
3,3510,Swedish Oatmeal Pancakes
4,4053,Fancy Feast Appetizers White Meat Chicken & Fl...
5,5777,Ritz Bacon Crackers
6,5890,Kind Organics Plant Calcium
7,6099,Keurig Nantucket Blend Medium Roast Coffee K Cups
8,6829,Quest Bar Chocolate Chip Cookie Dough Protein Bar
9,8272,"Refried Beans, with Jalapenos"
